In [1]:
%matplotlib inline
%pylab inline
import csv, time
from itertools import groupby
import os,sys
import astropy.io.ascii as ascii
from astropy.table import Table
from isochrones import StarModel, get_ichrone, SingleStarModel
import numpy as np
import pandas as pd
from astroquery.simbad import Simbad
from astropy import coordinates 
import astropy.units as u
#from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astroquery.gaia import Gaia
Simbad.add_votable_fields('sptype','flux(V)','flux(G)','flux(B)','distance','plx','plx_error','ids')

def GetGAIAData(GaiaDR3SourceIDs):
    # gets the GAIA data for the provided GaiaDR3SourceIDs's
 
    qry = "SELECT * FROM gaiadr3.gaia_source gs WHERE gs.source_id in (" + GaiaDR3SourceIDs + ");"
    
    job = Gaia.launch_job_async( qry )
    tblGaia = job.get_results()       #Astropy table
    return tblGaia
spec_st_param = np.genfromtxt("parameters.csv", dtype=None, delimiter=',', names=True ) 
#spec_st_param.dtype.names
#Simbad.list_votable_fields()

Populating the interactive namespace from numpy and matplotlib


Holoviews not imported. Some visualizations will not be available.
/tmp/ipykernel_1534257/1793534876.py:26: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  spec_st_param = np.genfromtxt("parameters.csv", dtype=None, delimiter=',', names=True )


In [2]:
### HARPS_RVBank ver02
st_name = spec_st_param['Name']

teff    = spec_st_param['teff']
e_teff  = spec_st_param['eteff']

logg    = spec_st_param['logg']
e_logg  = spec_st_param['elogg']
 
feh     = spec_st_param['F']
e_feh   = spec_st_param['eF']

In [6]:
spec_st_param_df = pd.DataFrame(spec_st_param)

In [7]:
spec_st_param_df

,Name,ra,dec,teff,eteff,logg,elogg,F,eF,vsini,evsini,RHKp,dRHKp,vsinflag,snr,RVflag,metflag
0,b'BD+012494',163.032119,0.493290,5524.2,50.000,4.113,0.080,0.29,0.042,4.045,0.263,0.000011,5.633458e-07,0,82.609042,0,0
1,b'BD+01316',26.633710,2.698870,6399.4,50.000,4.246,0.059,0.26,0.047,5.479,0.403,0.000007,3.591188e-07,0,350.859462,0,0
2,b'BD+034525',318.473940,4.452930,5753.9,61.286,4.099,0.126,0.05,0.050,3.934,0.270,0.000088,1.605549e-05,0,22.350000,0,0
3,b'BD+053805',278.947839,5.337380,5162.9,50.000,3.022,0.073,-0.09,0.043,4.215,0.446,0.000006,6.871098e-07,0,67.463379,0,0
4,b'BD+053818',279.139796,5.210510,4697.1,50.000,2.408,0.092,-0.10,0.043,4.245,0.573,0.000005,3.450043e-07,0,108.575000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,b'WASP-76',26.633048,2.700150,6432.4,50.000,4.313,0.058,0.28,0.048,5.495,0.433,0.000008,3.251607e-07,0,551.993258,0,0
4357,b'WASP-8',359.902750,-35.031210,5604.3,50.000,4.439,0.052,0.22,0.044,3.616,0.234,0.000027,5.197646e-07,0,372.505000,0,0
4358,b'WASP-80',303.166982,-2.145135,4838.3,170.327,2.884,0.400,-0.75,0.048,2.771,1.652,0.000198,1.077836e-04,0,72.411879,0,0
4359,b'WASP-89',313.899732,-18.971290,5099.2,50.000,4.384,0.084,-0.07,0.045,3.418,0.536,0.000044,1.263783e-06,0,41.675346,0,0


In [8]:
spec_st_param_df.shape

(4361, 17)

In [9]:
spec_st_param_df.columns.to_list()

['Name',
 'ra',
 'dec',
 'teff',
 'eteff',
 'logg',
 'elogg',
 'F',
 'eF',
 'vsini',
 'evsini',
 'RHKp',
 'dRHKp',
 'vsinflag',
 'snr',
 'RVflag',
 'metflag']

In [14]:
bp   = []
rp   = []
ag   = []
ID   = []
px   = []
e_px = []

for d in range(len(st_name[0:3])):

    print(st_name[d].decode("utf-8"))
    try:
        #### Simbad 
        result_table = Simbad.query_object(st_name[d].decode("utf-8"), wildcard=True)
        
        Ids = Simbad.query_objectids(st_name[d].decode("utf-8"), cache=False)
        Gaia_DR3_id = [i for i in Ids["ID"] if i.startswith('Gaia DR3')][0] #Gets the DR3 ID
        
        ##### Gaia  
        Gaia_table = GetGAIAData(Gaia_DR3_id[8:])
 
        bp.append(Gaia_table['phot_bp_mean_mag'][0])
        rp.append(Gaia_table['phot_rp_mean_mag'][0])
        ID.append(Gaia_DR3_id[8:])
        ag.append(Gaia_table['ag_gspphot'][0]) #https://arxiv.org/pdf/2206.07937.pdf
        px.append(Gaia_table['parallax'][0])
        e_px.append(Gaia_table['parallax_error'][0])
    except:    
        bp.append(np.nan)
        rp.append(np.nan)
        ID.append(np.nan)
        ag.append(np.nan)
        px.append(np.nan)
        e_px.append(np.nan)
        

BD+012494
INFO: Query finished. [astroquery.utils.tap.core]
BD+01316
INFO: Query finished. [astroquery.utils.tap.core]
BD+034525
INFO: Query finished. [astroquery.utils.tap.core]


In [18]:
Gaia_table.dtype.names

('solution_id',
 'DESIGNATION',
 'source_id',
 'random_index',
 'ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'parallax',
 'parallax_error',
 'parallax_over_error',
 'pm',
 'pmra',
 'pmra_error',
 'pmdec',
 'pmdec_error',
 'ra_dec_corr',
 'ra_parallax_corr',
 'ra_pmra_corr',
 'ra_pmdec_corr',
 'dec_parallax_corr',
 'dec_pmra_corr',
 'dec_pmdec_corr',
 'parallax_pmra_corr',
 'parallax_pmdec_corr',
 'pmra_pmdec_corr',
 'astrometric_n_obs_al',
 'astrometric_n_obs_ac',
 'astrometric_n_good_obs_al',
 'astrometric_n_bad_obs_al',
 'astrometric_gof_al',
 'astrometric_chi2_al',
 'astrometric_excess_noise',
 'astrometric_excess_noise_sig',
 'astrometric_params_solved',
 'astrometric_primary_flag',
 'nu_eff_used_in_astrometry',
 'pseudocolour',
 'pseudocolour_error',
 'ra_pseudocolour_corr',
 'dec_pseudocolour_corr',
 'parallax_pseudocolour_corr',
 'pmra_pseudocolour_corr',
 'pmdec_pseudocolour_corr',
 'astrometric_matched_transits',
 'visibility_periods_used',
 'astrometric_sigma5d_max

In [16]:
#Ids = Simbad.query_objectids('BD+072474', cache=False)
#Ids

In [17]:
#Gaia_table.dtype.names



In [18]:

# spec_st_param_df['eep_calc'] = np.nan
# spec_st_param_df['eep_e_calc'] = np.nan
# spec_st_param_df['age_calc']= np.nan
# spec_st_param_df['age_e_calc'] = np.nan
# spec_st_param_df['feh_calc'] = np.nan
# spec_st_param_df['feh_e_calc'] = np.nan
# spec_st_param_df['mass_calc'] = np.nan
# spec_st_param_df['mass_e_calc'] = np.nan
# spec_st_param_df['initial_mass_calc']= np.nan
# spec_st_param_df['initial_mass_e_calc'] = np.nan
# spec_st_param_df['radius_calc'] = np.nan
# spec_st_param_df['radius_e_calc'] = np.nan
# spec_st_param_df['density_calc'] = np.nan
# spec_st_param_df['density_e_calc'] = np.nan
# spec_st_param_df['logTeff_calc']= np.nan
# spec_st_param_df['logTeff_e_calc'] = np.nan
# spec_st_param_df['Teff_calc'] = np.nan
# spec_st_param_df['Teff_e_calc'] = np.nan
# spec_st_param_df['logg_calc'] = np.nan
# spec_st_param_df['logg_e_calc'] = np.nan
# spec_st_param_df['logL_calc']= np.nan
# spec_st_param_df['logL_e_calc'] = np.nan
# spec_st_param_df['Mbol_calc'] = np.nan
# spec_st_param_df['Mbol_e_calc'] = np.nan
# spec_st_param_df['delta_nu_calc']= np.nan
# spec_st_param_df['delta_nu_e_calc'] = np.nan
# spec_st_param_df['nu_max_calc'] = np.nan
# spec_st_param_df['nu_max_e_calc'] = np.nan
# spec_st_param_df['phase_calc'] = np.nan
# spec_st_param_df['phase_e_calc'] = np.nan
# spec_st_param_df['dm_deep_calc']= np.nan
# spec_st_param_df['dm_deep_e_calc'] = np.nan
# spec_st_param_df['BP_mag_calc'] = np.nan
# spec_st_param_df['BP_mag_e_calc'] = np.nan
# spec_st_param_df['RP_mag_calc']= np.nan
# spec_st_param_df['RP_mag_e_calc'] = np.nan
# spec_st_param_df['parallax_calc'] = np.nan
# spec_st_param_df['parallax_e_calc'] = np.nan
# spec_st_param_df['distance_calc'] = np.nan
# spec_st_param_df['distance_e_calc'] = np.nan
# spec_st_param_df['AV_calc']= np.nan
# spec_st_param_df['AV_e_calc'] = np.nan


In [24]:
check_params = {'Teff': (teff[0],e_teff[0]), 'logg': (logg[0], e_logg[0]), 'feh': (feh[0], e_feh[0]),'BP': (bp[0], 0.001), 
              'RP': (rp[0], 0.001),'parallax': (px[0], e_px[0])}#, 'AV': (0.101,0.009)} 


In [35]:
mist = get_ichrone('mist', bands=['BP', 'RP'])
os.system("export OMP_NUM_THREADS=10")

0

In [36]:
def calc_st_params_iso(i):

    print("TEST %s"%i)
    
    all_params = {'Teff': (teff[i],e_teff[i]), 'logg': (logg[i], e_logg[i]), 'feh': (feh[i], e_feh[i]),'BP': (bp[i], 0.001), 
              'RP': (rp[i], 0.001),'parallax': (px[i], e_px[i])}#, 'AV': (0.101,0.009)} 

    params = {}
    for key,value in all_params.items():
        if all(el is not None for el in value):
            params[key] = value
            print("Adding {} to table".format(key))
        else:
            print("Param NA")
        
    mod = SingleStarModel(mist,**params)        

    thread.map(mod.fit(n_live_points=1000,
    basename="%s"%st_name[i],
    verbose=False,
    refit=True,
    overwrite=True))

    derived_samples = mod.derived_samples.describe()

    
    spec_st_param_df['eep_calc'].iloc[i] = derived_samples['eep'][1]
    spec_st_param_df['eep_e_calc'].iloc[i] = derived_samples['eep'][2]
    spec_st_param_df['age_calc'].iloc[i] = derived_samples['age'][1]
    spec_st_param_df['age_e_calc'].iloc[i] = derived_samples['age'][2]
    spec_st_param_df['feh_calc'].iloc[i] = derived_samples['feh'][1]
    spec_st_param_df['feh_e_calc'].iloc[i] = derived_samples['feh'][2]
    spec_st_param_df['mass_calc'].iloc[i] = derived_samples['mass'][1]
    spec_st_param_df['mass_e_calc'].iloc[i] = derived_samples['mass'][2]
    spec_st_param_df['initial_mass_calc'].iloc[i] = derived_samples['initial_mass'][1]
    spec_st_param_df['initial_mass_e_calc'].iloc[i] = derived_samples['initial_mass'][2]
    spec_st_param_df['radius_calc'].iloc[i] = derived_samples['radius'][1]
    spec_st_param_df['radius_e_calc'].iloc[i] = derived_samples['radius'][2]
    spec_st_param_df['density_calc'].iloc[i] = derived_samples['density'][1]
    spec_st_param_df['density_e_calc'].iloc[i] = derived_samples['density'][2]
    spec_st_param_df['logTeff_calc'].iloc[i] = derived_samples['logTeff'][1]
    spec_st_param_df['logTeff_e_calc'].iloc[i] = derived_samples['logTeff'][2]
    spec_st_param_df['Teff_calc'].iloc[i] = derived_samples['Teff'][1]
    spec_st_param_df['Teff_e_calc'].iloc[i] = derived_samples['Teff'][2]
    spec_st_param_df['logg_calc'].iloc[i] = derived_samples['logg'][1]
    spec_st_param_df['logg_e_calc'].iloc[i] = derived_samples['logg'][2]
    spec_st_param_df['logL_calc'].iloc[i] = derived_samples['logL'][1]
    spec_st_param_df['logL_e_calc'].iloc[i] = derived_samples['logL'][2]
    spec_st_param_df['Mbol_calc'].iloc[i] = derived_samples['Mbol'][1]
    spec_st_param_df['Mbol_e_calc'].iloc[i] = derived_samples['Mbol'][2]
    spec_st_param_df['delta_nu_calc'].iloc[i] = derived_samples['delta_nu'][1]
    spec_st_param_df['delta_nu_e_calc'].iloc[i] = derived_samples['delta_nu'][2]
    spec_st_param_df['nu_max_calc'].iloc[i] = derived_samples['nu_max'][1]
    spec_st_param_df['nu_max_e_calc'].iloc[i] = derived_samples['nu_max'][2]
    spec_st_param_df['phase_calc'].iloc[i] = derived_samples['phase'][1]
    spec_st_param_df['phase_e_calc'].iloc[i] = derived_samples['phase'][2]
    spec_st_param_df['dm_deep_calc'].iloc[i] = derived_samples['dm_deep'][1]
    spec_st_param_df['dm_deep_e_calc'].iloc[i] = derived_samples['dm_deep'][2]
    spec_st_param_df['BP_mag_calc'].iloc[i] = derived_samples['BP_mag'][1]
    spec_st_param_df['BP_mag_e_calc'].iloc[i] = derived_samples['BP_mag'][2]
    spec_st_param_df['RP_mag_calc'].iloc[i] = derived_samples['RP_mag'][1]
    spec_st_param_df['RP_mag_e_calc'].iloc[i] = derived_samples['RP_mag'][2]
    spec_st_param_df['parallax_calc'].iloc[i] = derived_samples['parallax'][1]
    spec_st_param_df['parallax_e_calc'].iloc[i] = derived_samples['parallax'][2]
    spec_st_param_df['distance_calc'].iloc[i] = derived_samples['distance'][1]
    spec_st_param_df['distance_e_calc'].iloc[i] = derived_samples['distance'][2]
    spec_st_param_df['AV_calc'].iloc[i] = derived_samples['AV'][1]
    spec_st_param_df['AV_e_calc'].iloc[i] = derived_samples['AV'][2]    
    return

In [37]:
from pathos.pools import ProcessPool as Pool
#from pathos.threading import ThreadPool as Pool

def partial_func(samp_ii):
    loglik = calc_st_params_iso(samp_ii)
    return loglik

with Pool(ncpus=10) as thread: 
    results = thread.map(partial_func,   [samp_i for samp_i in range(len(st_name[0:3]))])
    
    print(results)

thread.close()
thread.join()
thread.clear()

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points = 1000
 dimensionality =    5
 *****************************************************
 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points = 1000
 dimensionality =    5
 *****************************************************
 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points = 1000
 dimensionality =    5
 *****************************************************

 MultiNest Warning!
 Parameter            4  of mode            1  is converging towards the edge of the 

TypeError: map() requires at least two args

In [ ]:
spec_st_param_df.Teff_calc[3]